In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
import modules.data_prep as data_prep
import modules.visualisation as viz
from modules.model import EarthquakeAnalyzer
from modules.config import START_MONTH, START_YEAR, END_MONTH, END_YEAR, TUPLE_COLUMNS_TO_UNPACK, HIGH_MAG_THRESHOLD

## Data Prep

In [ ]:
data, filtered_features, gj = data_prep.data_prep_pipeline()

## Save the map

In [ ]:
data = data_prep.re_filter_data_by_date_interval(data) 
viz.map_maker(data, filtered_features, gj)